In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from typing import Union

import numpy as np
import pandas as pd


def gaussian_weight(
    age: Union[float, pd.Series, np.ndarray],
    mean: float,
    sigma: float,
    epsilon: float = 0.01
):
    """Calculate Gaussian kernel weight for brain scan observed at specified age."""
    w = np.exp(- 0.5 * np.power((age - mean) / sigma, 2))
    w /= sigma * np.sqrt(2.0 * np.pi)
    w[w < epsilon] = 0.0
    return w


def plot_age_at_scan_difference(ax, data: pd.DataFrame, mean: float, sigma: float, epsilon: float = 0.01):
    """Plot distribution of time-to-scan for samples with non-zero weight in given temporal kernel support region."""
    df = data[gaussian_weight(age=data.scan_age, mean=mean, sigma=sigma, epsilon=epsilon) > 0]
    (df.scan_age - mean).hist(ax=ax, bins=np.arange(-5, 5.01, 0.25))
    ax.set_title("mean={:.0f}, sigma={:.2f}, N={:d}".format(mean, sigma, len(df)))
    ax.set_xticks(np.arange(-4, 4.01, 1))

In [ ]:
participants = pd.read_csv("participants.tsv", sep="\t")
participants.rename(columns={"age_at_scan": "scan_age", "birth_ga": "birth_age"}, inplace=True)
participants["time_to_scan"] = participants.scan_age - participants.birth_age
participants.sort_values(by=["participant_id", "session_id", "time_to_scan", "scan_age"], inplace=True)
participants.head()

In [ ]:
neonatal_scans = participants[(participants.time_to_scan >= 0.0)]
selected_scans = neonatal_scans[(neonatal_scans.time_to_scan < 4.0) & ((neonatal_scans.score < 1) | (neonatal_scans.score > 2))]

subjects = participants.drop_duplicates(subset="participant_id", keep="first")
neonates = neonatal_scans.drop_duplicates(subset="participant_id", keep="first")
selected = selected_scans.drop_duplicates(subset="participant_id", keep="first")

print("No. of brain scans:    {:3d} ({:3d} subjects)".format(len(participants), len(subjects)))
print("No. of neonatal scans: {:3d} ({:3d} subjects)".format(len(neonatal_scans), len(neonates)))
print("No. of selected scans: {:3d} ({:3d} subjects)".format(len(selected_scans), len(selected)))
print()
print("Minimum neonatal brain age at scan: {}".format(neonates.scan_age.min()))
print("Maximum neonatal brain age at scan: {}".format(neonates.scan_age.max()))
print()
print("Minimum selected brain age at scan: {}".format(selected.scan_age.min()))
print("Maximum selected brain age at scan: {}".format(selected.scan_age.max()))

gender_counts = dict(neonates.gender.value_counts())
print()
print("No. of selected male scans:   {:3d}".format(gender_counts["Male"]))
print("No. of selected female scans: {:3d}".format(gender_counts["Female"]))

print()
for cnt, wk in zip(*np.histogram(selected.scan_age, bins=np.arange(26.0, 45.0))):
    print("Week {:2.0f}: {:2d} selected scans".format(wk, cnt))

print()
for wk in [30, 31, 32, 33, 34, 35, 36]:
    m = len(neonates[(neonates.scan_age <= wk)])
    n = len(selected[(selected.scan_age <= wk)])
    print("No. of brain scans before week {}: {} ({} selected)".format(wk, m, n))

In [ ]:
selected.scan_age.hist(bins=np.arange(26.0, 45.5, 0.05), figsize=(20, 6));

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 4), sharex=True, sharey=True)
for ax, df in zip(axes.flat, [neonates, selected]):
    df.scan_age.hist(ax=ax, bins=np.arange(26.0, 36.5, 0.1))
fig;

# Subsample term scans

In [ ]:
df = selected

delta = 0.1
n_samples = 20

prng = np.random.RandomState(seed=0)
for t in np.arange(df.scan_age.min(), df.scan_age.max() + 0.001, delta):
    a = df[(df.scan_age - t).abs() < delta]
    if len(a):
        b = a.sample(n=n_samples, random_state=prng, replace=True)
        df = df[df.participant_id.isin(b.participant_id) | ~df.participant_id.isin(a.participant_id)]

dataset = df

print("No. of remaining brain scans:", len(dataset))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 6), sharex=True, sharey=True)
selected.scan_age.hist(ax=axes[0], bins=np.arange(26.0, 45.5, 0.1))
dataset.scan_age.hist(ax=axes[1], bins=np.arange(26.0, 45.5, 0.1));

In [ ]:
df = pd.DataFrame(data={"subject_id": neonates.participant_id + "-" + neonates.session_id.astype(str), "age": neonates.scan_age})
df.to_csv("ages.csv", index=False, header=False)

df = pd.DataFrame(data={"subject_id": dataset.participant_id + "-" + dataset.session_id.astype(str)})
df.to_csv("subjects.lst", index=False, header=False)

# Find adaptive kernel widths

In [ ]:
means = np.arange(29.0, 45.0, 1.0)

Determine median number of subjects per time point given target `sigma` value.

In [ ]:
counts = {}
median = {}
sigmas = [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
for sigma in sigmas:
    n_nonzero_weights = []
    for mean in means:
        weights = gaussian_weight(dataset.scan_age, mean, sigma)
        n_nonzero_weights.append(np.count_nonzero(weights))
    counts[sigma] = np.array(n_nonzero_weights).astype(int)
    median[sigma] = np.median(counts[sigma]).astype(int)
    print("constant sigma: {:3.2f}, counts: {}, median: {:3d}".format(sigma, counts[sigma], median[sigma]))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 6))

xmin = means[0] - 1
xmax = means[-1] + 1

sigmas = [0.5, 1.0, 1.5, 2.0]
for sigma in sigmas:
    ax.scatter(means, counts[sigma], label="sigma={:0.2f}".format(sigma))
    ax.plot([xmin, xmax], [median[sigma]] * 2)

ax.plot([xmin, xmax], [20] * 2, color='lightgray')

ax.set_xlim([xmin, xmax])
ax.set_xticks(means)
ax.set_xlabel("atlas time point [PMA]")

ax.set_yticks([20] + [median[sigma] for sigma in sigmas])
ax.set_ylabel("images with non-zero weight [N]")

ax.legend(loc="upper left")

#fig.savefig("/Users/aschuh/Desktop/dhcp_atlas_no_images_for_varying_constant_sigma_values.png")

fig;

In [ ]:
target_sigma = 1.0
target_count = 100

step_size = 0.01
min_sigma = 0.2
max_sigma = 1.5


def count_nonzero_weights(dataset, means, sigmas):
    nz = []
    for mean, sigma in zip(means, sigmas):
        weights = gaussian_weight(dataset.scan_age, mean, sigma)
        nz.append(np.count_nonzero(weights))
    return np.array(nz).astype(int)


sigmas = np.array([target_sigma] * len(means))
for _ in range(100):
    nz = count_nonzero_weights(dataset, means, sigmas)
    sigmas[nz < 0.98 * target_count] += step_size
    sigmas[nz > 1.02 * target_count] -= step_size

nz = count_nonzero_weights(dataset, means, sigmas)
sigmas[nz < 0.98 * target_count] += step_size

sigmas = np.clip(sigmas, min_sigma, max_sigma)
nz = count_nonzero_weights(dataset, means, sigmas)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 6))

xmin = means[0] - 1
xmax = means[-1] + 1

median_count = np.median(nz)

ax.scatter(means, nz)
#ax.plot([xmin, xmax], [median_count] * 2)

yticks = list(range(50, 310, 50))
for n in yticks:
    ax.plot([xmin, xmax], [n, n], color='lightgray')

ax.set_xlim([xmin, xmax])
ax.set_xticks(means)
ax.set_xlabel("atlas time point [PMA]")

ax.set_ylim([0, 300])
ax.set_yticks(yticks)
ax.set_ylabel("images with non-zero weight [N]")

#fig.savefig("/Users/aschuh/Desktop/dhcp_atlas_no_images_for_adaptive_sigma_values_target_count_100_maxsigma_2.png")

fig;

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(20, 12), sharex=True, sharey=True)

for i, (mean, sigma) in enumerate(zip(means, sigmas)):
    plot_age_at_scan_difference(axes.flat[i], data=dataset, mean=mean, sigma=sigma)

for ax in axes[-1,:]:
    ax.set_xlabel("age at scan offset [PMA]", labelpad=8)
for ax in axes[:,0]:
    ax.set_ylabel("no. of brain scans [N]", labelpad=10)

In [ ]:
import os

os.makedirs("weights", exist_ok=True)

df = pd.DataFrame(data={
    "t": ["{:.0f}".format(mean) for mean in means],
    "sigma": ["{:.2f}".format(sigma) for sigma in sigmas]
})
df.to_csv("weights/sigmas.csv", index=False)

for mean, sigma in zip(means, sigmas):
    weights = gaussian_weight(dataset.scan_age, mean, sigma)
    df = pd.DataFrame(data={
        "subject_id": dataset.participant_id + "-" + dataset.session_id.astype(str),
        "weight": weights
    })
    df= df[df.weight > 0].sort_values(by="weight", ascending=False)
    df.to_csv("weights/t{:.0f}.tsv".format(mean), index=False, header=False, sep="\t")